#### First Let's understand the basic Catalog + Volume Feature of Databricks 
Filesystem Hierarchy of Volume in Databricks (DBFS)?<br>
Catalog -> /OurWorkspace/catalog/schema(database)/volume/folder/data files<br>
Tables Hierarchy of Databricks?<br>
Catalog -> /OurWorkspace/catalog/schema(database)/tables/data(dbfs filesystem/some other filesystems)<br>

In [0]:
%sql
create catalog if not exists catalog1_dropme;
create database if not exists catalog1_dropme.schema1_dropme;
create volume if not exists catalog1_dropme.schema1_dropme.volume1_dropme;

In [0]:
dbutils.fs.mkdirs("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme")
#Upload the drive data into this location...

####If we need to create schema/volume/folder programatically, follow the below steps

#Spark SQL<br>
###1.E(Extract) 
L(Load)<br>
Inbuilt libraries sources/targets & Inbuilt data Formats<br>
2. Bread & Butter (T(Transformation) A(Analytical))<br>

In [0]:
dbutils.fs.mkdirs("/Volumes/workspace/wd36schema2/volume1/folder1")

#Learn How to Create Dataframes from filesystem using different options
Download the data from the below drive url <br>
https://drive.google.com/drive/folders/1Tw7V9eBtUxy0xQMW38z3-bzWI_ewzLm6?usp=drive_link

###How Do We Write a Typical Spark Application (Core(Obsolute),SQL(Important),Streaming(Mid level important))
####Before we Create Dataframe/RDD, what is the prerequisite? We need to create spark session object by instantiating sparksession class (by default databricks did that if you create a notebook)

In [0]:
from pyspark.sql.session import SparkSession
print(spark)#already instantiated by databricks
spark1=SparkSession.builder.getOrCreate()
print(spark1)#we instantiated

Create a DBFS volume namely commondata and upload the above data in that volume
What are other FS uri's available? file:///, hdfs:///, dbfs:///, gs:///, s3:///, adls:///, blob:///

####How to Read/Extract the data from the filesytem and load it into the distributed memory for further processing/load - using diffent methodologies/options from different sources(fs & db) and different builtin formats (csv/json/orc/parquet/delta/tables)

In [0]:

#If I don't use any options in this csv function, what is the default functionality?
#1. By default it will consider ',' as a delimiter (sep='~')
#2. By default it will use _c0,_c1..._cn it will apply as column headers (header=True or toDF("","","") or we have more options to see further)
#3. By default it will treat all columns as string (inferSchema=True or we have more options to see further)
csv_df1=spark.read.csv("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_1")
#csv_df1.show(2)#display with produce output in a dataframe format
print(csv_df1.printSchema())
display(csv_df1)#display with produce output in a beautified table format, specific to databricks


csv_df1=spark.read.csv
(path: PathOrPaths, schema: Optional[Union[StructType, str]]=None, sep: Optional[str]=None, encoding: Optional[str]=None, quote: Optional[str]=None, escape: Optional[str]=None, comment: Optional[str]=None, header: Optional[Union[bool, str]]=None, inferSchema: Optional[Union[bool, str]]=None, ignoreLeadingWhiteSpace: Optional[Union[bool, str]]=None, ignoreTrailingWhiteSpace: Optional[Union[bool, str]]=None, nullValue: Optional[str]=None, nanValue: Optional[str]=None, positiveInf: Optional[str]=None, negativeInf: Optional[str]=None, dateFormat: Optional[str]=None, timestampFormat: Optional[str]=None, maxColumns: Optional[Union[int, str]]=None, maxCharsPerColumn: Optional[Union[int, str]]=None, maxMalformedLogPerPartition: Optional[Union[int, str]]=None, mode: Optional[str]=None, columnNameOfCorruptRecord: Optional[str]=None, multiLine: Optional[Union[bool, str]]=None, charToEscapeQuoteEscaping: Optional[str]=None, samplingRatio: Optional[Union[float, str]]=None, enforceSchema: Optional[Union[bool, str]]=None, emptyValue: Optional[str]=None, locale: Optional[str]=None, lineSep: Optional[str]=None, pathGlobFilter: Optional[Union[bool, str]]=None, recursiveFileLookup: Optional[Union[bool, str]]=None, modifiedBefore: Optional[Union[bool, str]]=None, modifiedAfter: Optional[Union[bool, str]]=None, unescapedQuoteHandling: Optional[str]=None) -> "DataFrame"

Sample data of custs_1
4000001,Kristina,Chung,55,Pilot<br>
4000002,Paige,Chen,77,Teacher

Sample data of custs_header<br>
custid,fname,lname,age,profession<br>
4000001,Kristina,Chung,55,Pilot<br>
4000002,Paige,Chen,77,Teacher

In [0]:
#1. Header Concepts (Either we have define the column names or we have to use the column names from the data)
#Important option is...
#By default it will use _c0,_c1..._cn it will apply as column headers, but we are asking spark to take the first row as header and not as a data?
csv_df1=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header",header=True)
print(csv_df1.printSchema())
csv_df1.show(2)
#csv_df1.write.csv("/Volumes/workspace/wd36schema2/volume1/folder1/outputdata")
#By default it will use _c0,_c1..._cn it will apply as column headers, if we use toDF(colnames) we can define our own headers..
csv_df2=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1").toDF("id","fname","lname","age","prof")
#csv_df1.show(2)#display with produce output in a dataframe format
print(csv_df2.printSchema())
csv_df2.show(2)

In [0]:
#2. Printing Schema (equivalent to describe table)
csv_df1.printSchema()
csv_df2.printSchema()

In [0]:
#3. Inferring Schema 
# (Performance Consideration: Use this function causiously because it scans the entire data by immediately evaluating and executing
# hence, not good for large data or not good to use on the predefined schema dataset)
#sample data
#4004979,Tara,Drake,32,
#4004980,Earl,Hahn,34,Human resources assistant
#4004981,Don,Jones,THIRTY SIX,Lawyer
csv_df1=spark.read.csv("/Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/custs_2.txt",inferSchema=True).toDF("id","fname","lname","age","prof")
csv_df1.where("id in (4004979,4004981)").show(2)
csv_df1.printSchema()

In [0]:
#4. Using delimiter or seperator option
csv_df1=spark.read.csv("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust_other_del.txt",header=True,sep='~')
csv_df1.show(2)
csv_df1.printSchema()

In [0]:
#5. Using different options to create dataframe with csv and other module... (2 methodologies with 3 ways of creating dataframes)
csv_df1=spark.read.csv("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust_other_del.txt",inferSchema=True,header=True,sep='~')
csv_df1.show(2)
#or another way of creating dataframe (from any sources whether builtin or external)...
#option can be used for 1 or 2 option...
csv_df2=spark.read.option("header","True").option("inferSchema","true").option("sep","~").format("csv").load("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust_other_del.txt")
csv_df2.show(2)
#options can be used for multiple options in one function as a parameter...
csv_df3=spark.read.options(header="True",inferSchema="true",sep="~").format("csv").load("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust_other_del.txt")
csv_df3.show(2)

### Generic way of read and load data into dataframe using fundamental options from built in sources (csv/orc/parquet/xml/json/table) (inferschema, header, sep)

In [0]:
csv_df1=spark.read.csv("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust_other_del.txt",inferSchema=True,header=True,sep='~')
csv_df1.show(2)

### Generic way of read and load data into dataframe using extended options from external sources (bigquery/redshift/athena/synapse) (tmpfolder, access controls)

In [0]:
#options can be used for multiple options in one function as a parameter...
csv_df3=spark.read.options(header="True",inferSchema="true",sep="~").format("csv").load("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust_other_del.txt")
csv_df3.show(2)

### Reading data from multiple files

In [0]:
csv_df1=spark.read.csv("dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust*",inferSchema=True,header=True,sep='~')
print(csv_df1.count())

In [0]:
csv_df1=spark.read.csv(path=["dbfs:///Volumes/catalog1_dropme/schema1_dropme/volume1_dropme/directory_dropme/cust*","/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1"],inferSchema=True,header=True,sep=',')
print(csv_df1.count())

### Provide schema with SQL String or programatically (very very important)

[PySpark SQL Datatypes](https://spark.apache.org/docs/latest/sql-ref-datatypes.html) <br>
[Data Types](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/data_types.html)